In [280]:
import pandas as pd
import numpy as np
from utils import get_species, get_labels, get_labels_all
from utils import get_taxonomy

In [281]:
X, y, y_all = get_taxonomy(), get_labels(), get_labels_all()

In [282]:
raw = pd.read_csv("../data/raw.csv", index_col=[1, 4])

/Users/daniel/opt/anaconda3/envs/data/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (11,12,13,15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [283]:
raw.shape

(12532, 3217)

In [284]:
# to reduce batch effects
non_illumina = [
    "454 GS FLX Titanium",
    "Ion Torrent PGM",
    "Ion Torrent Proton",
    "BGISEQ-500",
]

In [285]:
remove = (
    (y_all == "Underweight").values.flatten() | 
    (y_all == "Overweight").values.flatten() |
    (y_all == "Obesity").values.flatten() |
    (y_all == "Obese").values.flatten() |
    raw["Sequencing Platform"].isin(non_illumina).values.flatten() |
    (y_all.index.get_level_values(0) == "P4") | # P4 treats the poop for extracting viral DNA
    (y_all.index.get_level_values(0) == "P86") | # Healthy at baseline but half develop T2D 
#     y2.values.flatten() | 
    ((y_all.index.get_level_values(0) == "P48") & (y_all == "Healthy").values.flatten()) | # Alcohol or smoking
    (y_all.index.get_level_values(0) == "P59") | # Are all technically healthy, but half are in heavily urbanized areas
    # and "Microbes with higher relative abundance in Chinese urban samples have been associated with disease in other studies"
    (y_all.index.get_level_values(0) == "P63") | # Deals with semisupercentenarians, i.e., 105 to 109 years old
#     (y_all.index.get_level_values(0) == "GMHI-4") |
    (X['UNKNOWN'] >= 100).values.flatten()
    )

In [286]:
X, y, y_all = X.iloc[~remove, :], y.iloc[~remove, :], y_all.iloc[~remove, :]

In [287]:
raw = raw.iloc[~remove, :]

In [288]:
X = X.divide((100 - X["UNKNOWN"]), axis="rows")

In [289]:
remove

array([False, False, False, ...,  True,  True,  True])

In [290]:
X.shape

(9067, 3200)

In [291]:
len(np.unique(X.index.get_level_values(0)))

62

In [292]:
from sklearn.preprocessing import OrdinalEncoder

sample_studies = np.array(X.index.get_level_values(0))
o = OrdinalEncoder()
groups = o.fit_transform(sample_studies.reshape((len(sample_studies), 1))).flatten()
groups

array([ 9.,  9.,  9., ..., 34., 34., 34.])

In [293]:
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GroupKFold
logo = LeaveOneGroupOut()
kfold = StratifiedKFold(10, shuffle=True, random_state=42)
gkfold = GroupKFold(10)

In [294]:
from sklearn.base import BaseEstimator

class GMHI_model(BaseEstimator):

    def __init__(self, use_shannon=True, theta_f=1.4, theta_d=0.1):
        self.use_shannon = use_shannon
        self.thresh = 0.00001
        self.health_abundant = None
        self.health_scarce = None
        self.features = None
        self.theta_f = theta_f
        self.theta_d = theta_d
        self.fitted = False

    def fit(self, X, y):
        """
        Identifies health_abundant and health_scarce
        columns/features
        """
        self.features = X.columns
        self.classes_ = np.unique(y)
        if(isinstance(X, pd.DataFrame)):
            X = X.values
        if(isinstance(y, pd.DataFrame)):
            y = y.values
        difference, fold_change = self.get_proportion_comparisons(X, y)
        self.select_features(difference, fold_change)
        self.fitted = True

    def get_proportion_comparisons(self, X, y):
        # get healthy and unhealthy samples
        healthies = X[y.flatten(), :]
        unhealthies = X[~y.flatten(), :]

        # get proportions for each species
        proportion_healthy = self.get_proportions(healthies)
        proportion_unhealthy = self.get_proportions(unhealthies)

        # get differences and fold change
        diff = proportion_healthy - proportion_unhealthy
        fold = proportion_healthy / proportion_unhealthy
        return diff, fold

    def get_proportions(self, samples_of_a_class):
        num_samples = samples_of_a_class.shape[0]
        p = np.sum(samples_of_a_class > self.thresh, axis=0) / num_samples
        return p

    def select_features(self, difference, fold_change):
        # based on proportion differences and fold change, select health abundant
        # and health scarce
        self.health_abundant = self.features[self.cutoff(difference, fold_change)]
        self.health_scarce = self.features[self.cutoff(-1 * difference, 1 / fold_change)]

    def cutoff(self, diff, fold):
        diff_cutoff = diff > self.theta_d
        fold_cutoff = fold > self.theta_f
        both_cutoff = np.bitwise_and(diff_cutoff, fold_cutoff)
        columns = np.where(both_cutoff)
        return columns[0]

    def decision_function(self, X):
        X_healthy_features = X[self.health_abundant]
        X_unhealthy_features = X[self.health_scarce]
        psi_MH = self.get_psi(X_healthy_features.values) / (
            X_healthy_features.shape[1])
        psi_MN = self.get_psi(X_unhealthy_features.values) / (
            (X_unhealthy_features.shape[1]))
        num = psi_MH + self.thresh
        dem = psi_MN + self.thresh
        return np.log10(num / dem)

    def get_psi(self, X):
        psi = self.richness(X) * 1.0
        if self.use_shannon:
            shan = self.shannon(X)
            psi *= shan
        return psi

    def richness(self, X):
        """
        Returns the number of nonzero values for each sample (row) in X
        """
        rich = np.sum(X > self.thresh, axis=1)
        return rich

    def shannon(self, X):
        logged = np.log(X)
        logged[logged == -np.inf] = 0
        logged[logged == np.inf] = 0
        shan = logged * X * -1
        return np.sum(shan, axis=1)

    def predict(self, X):
        return self.decision_function(X) > 0

In [295]:
def get_scores(gmhi2, y, threshold):
    y = y.iloc[(abs(gmhi2) >= threshold).values]
    gmhi2 = gmhi2.iloc[(abs(gmhi2) >= threshold).values]
    bal = balanced_accuracy_score(y, gmhi2 > 0)
    acc = accuracy_score(y, gmhi2 > 0)
    return {"n" : gmhi2.shape[0], "bal" : bal, "acc" : acc}

In [296]:
def cross_val_predict2(data, target, groups):
    clf = GMHI_model(theta_f=1.4, theta_d=0.1)
    predictions = np.zeros((target.shape[0],))
    for group in range(len(np.unique(groups))):
        X_train = data.iloc[groups != group, :]
        y_train = target.iloc[groups != group, :]
        X_test = data.iloc[groups == group, :]
        y_test = target.iloc[groups == group, :]
        clf.fit(X_train, y_train)
        pred = clf.decision_function(X_test)
        predictions[groups == group] = pred
    return predictions

In [193]:
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomTreesEmbedding
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import balanced_accuracy_score, accuracy_score

c = 0.00001

final_dic = []
Cs = [0.01, 0.02, 0.03, 0.1, 0.2, 0.3, 1, 2, 3]

for C in Cs
    clf = LogisticRegression(random_state=42, penalty="l1", solver="liblinear", C=C, class_weight="balanced")

    predictions = cross_val_predict(clf, X > c, y.values.flatten(), method="decision_function",
                             groups=groups, 
                                    cv=logo, verbose=2, 
                                    n_jobs=-1
                                   )
    pred = pd.DataFrame(predictions, index=y.index, columns=["gmhi2"])
    max_thresh = float(max(abs(pred.values)))
    for thresh in np.arange(0, max_thresh, 0.1):
        dic = get_scores(pred, y, thresh)
        dic["thresh"] = thresh
        dic["C"] = C
        final_dic.append(dic)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done  62 out of  62 | elapsed:   24.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done  62 out of  62 | elapsed:   32.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done  62 out of  62 | elapsed:   29.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    8.0s
[Parallel(n_jobs=-1)]: Done  62 out of  62 | elapsed:   30.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   10.1s
[Parallel(n_jobs=-1)]

In [194]:
df = pd.DataFrame(final_dic)

In [216]:
max(df["thresh"])

26.6

In [218]:
threshes = np.arange(0, max(df["thresh"]) + 0.1, 0.1)

In [219]:
final_dic

[{'n': 9067,
  'bal': 0.7320200116446521,
  'acc': 0.7330980478658873,
  'thresh': 0.0,
  'C': 0.01},
 {'n': 8409,
  'bal': 0.7459288483498565,
  'acc': 0.7469378047330242,
  'thresh': 0.1,
  'C': 0.01},
 {'n': 7774,
  'bal': 0.7622455270312102,
  'acc': 0.7630563416516594,
  'thresh': 0.2,
  'C': 0.01},
 {'n': 7158,
  'bal': 0.7759202598758044,
  'acc': 0.7757753562447611,
  'thresh': 0.30000000000000004,
  'C': 0.01},
 {'n': 6556,
  'bal': 0.7899163462094009,
  'acc': 0.7895057962172056,
  'thresh': 0.4,
  'C': 0.01},
 {'n': 5959,
  'bal': 0.8034599779137146,
  'acc': 0.8023158248028193,
  'thresh': 0.5,
  'C': 0.01},
 {'n': 5394,
  'bal': 0.8143676596202669,
  'acc': 0.8125695216907676,
  'thresh': 0.6000000000000001,
  'C': 0.01},
 {'n': 4814,
  'bal': 0.8288042027484437,
  'acc': 0.8265475695886996,
  'thresh': 0.7000000000000001,
  'C': 0.01},
 {'n': 4316,
  'bal': 0.8389239497976402,
  'acc': 0.8354958294717331,
  'thresh': 0.8,
  'C': 0.01},
 {'n': 3782,
  'bal': 0.851320550570

In [276]:
table = {}

for thresh in threshes:
    li = []
    for C in Cs:
        row = df[(df["thresh"] == thresh) & (df["C"] == C)]
        if row.shape[0] == 0:
            li.append(0)
        else:
            li.append(int(row["n"]))
    table[thresh] = li

In [277]:
exp = pd.DataFrame.from_dict(table, orient="index", columns=["C=" + str(c) for c in Cs])

In [278]:
exp

,C=0.01,C=0.02,C=0.03,C=0.1,C=0.2,C=0.3,C=1,C=2,C=3
0.0,9067,9067,9067,9067,9067,9067,9067,9067,9067
0.1,8409,8528,8595,8649,8660,8692,8756,8775,8822
0.2,7774,8013,8102,8225,8281,8348,8451,8521,8564
0.3,7158,7509,7627,7796,7917,7968,8145,8255,8302
0.4,6556,6966,7157,7384,7535,7605,7852,7977,8049
...,...,...,...,...,...,...,...,...,...
26.2,0,0,0,0,0,0,0,0,1
26.3,0,0,0,0,0,0,0,0,1
26.4,0,0,0,0,0,0,0,0,1
26.5,0,0,0,0,0,0,0,0,1


In [279]:
exp.to_csv("num_retained.csv")

In [337]:
clf = LogisticRegression(random_state=42, penalty="l1", solver="liblinear", C=0.1, class_weight="balanced")
clf.fit(X > c, y.values.flatten())
predictions = clf.predict(X > c)
balanced_accuracy_score(y, predictions), accuracy_score(y, predictions)

(0.8094430133998434, 0.8055586191684129)

In [343]:
coef = pd.DataFrame(clf.coef_[0], index=X.columns, columns=["Coefficient"])

In [348]:
coef_nonzero = coef[coef["Coefficient"] != 0]

In [349]:
coef_nonzero.to_csv("coefficients_nonzero.csv")

In [356]:
coef_nonzero.sort_values("Coefficient")[:30]

,Coefficient
k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Clostridiales_Family_XIII_Incertae_Sedis|g__Mogibacterium,-0.823169
k__Bacteria|p__Firmicutes|c__Negativicutes|o__Veillonellales|f__Veillonellaceae|g__Dialister|s__Dialister_pneumosintes,-0.683509
k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Lachnospiraceae|g__Lachnoclostridium,-0.538762
k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Eubacteriaceae|g__Eubacterium|s__Eubacterium_sp_CAG_180,-0.535096
k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactobacillales|f__Lactobacillaceae|g__Lactobacillus|s__Lactobacillus_amylovorus,-0.506429
k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactobacillales|f__Lactobacillaceae|g__Lactobacillus|s__Lactobacillus_salivarius,-0.489443
k__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Enterobacterales|f__Morganellaceae,-0.460433
k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Peptostreptococcaceae|g__Peptostreptococcus|s__Peptostreptococcus_stomatis,-0.456542
k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactobacillales|f__Lactobacillaceae|g__Lactobacillus|s__Lactobacillus_mucosae,-0.420368
k__Bacteria|p__Fusobacteria|c__Fusobacteriia|o__Fusobacteriales|f__Fusobacteriaceae|g__Fusobacterium|s__Fusobacterium_nucleatum,-0.406313


In [330]:
clf = LogisticRegression(random_state=42, penalty="l1", solver="liblinear", C=0.1, class_weight="balanced")

predictions = cross_val_predict(clf, X > c, y.values.flatten(), method="decision_function",
                         groups=groups, 
                                cv=logo, verbose=2, 
                                n_jobs=-1
                               )
pred = pd.DataFrame(predictions, index=y.index, columns=["gmhi2"])

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    8.3s
[Parallel(n_jobs=-1)]: Done  62 out of  62 | elapsed:   31.5s finished


In [332]:
thresh = 1.2
X_red = X[abs(pred["gmhi2"]) >= thresh]
y_red = y[abs(pred["gmhi2"]) >= thresh]

In [336]:
clf = LogisticRegression(random_state=42, penalty="l1", solver="liblinear", C=0.1, class_weight="balanced")
clf.fit(X_red > c, y_red.values.flatten())
predictions = clf.predict(X_red > c)
balanced_accuracy_score(y_red, predictions), accuracy_score(y_red, predictions)

(0.8704728497174274, 0.8712914485165794)